# 1. Extraction de Keywords

## Imports

In [ ]:
import os
import yake

## Extraire les mots clés d'un document avec Yake

https://github.com/LIAAD/yake

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:
# Lister les Fichiers
data_path = "../data/txt/"
files = [f for f in os.listdir(data_path) if f.endswith('.txt') and f.startswith('KB_JB838_1930')]


In [ ]:
for f in sorted(files)[:10]:
    text = open(os.path.join(data_path, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

# 2. Nuages de mots

## Imports et stopwords

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

In [ ]:
# Stopwords (Idem que dans s1)
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "rue", "rue", "dem", "rossel", "très", "bon",
       "ans", "place", "heures", "ecr", "avenue", "tél", "bonne", "cond", "brux", "vend",
       "midi", "belle", "adr", "toute", "etc", "bons", "grande", "grand", "demande",
       "nord", "jour", "jours", "ecrire", "mois", "soir", "cherche"]
sw = set(sw)


## Créer un fichier contenant le texte de tous les journaux d'une année donnée

In [ ]:
year = 1930

# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]
len(txts)

In [ ]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r', encoding='utf-8') as f:
        content_list.append(f.read())

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

## Nettoyer le fichier à l'aide d'une fonction de nettoyage

### Créer la fonction de nettoyage

In [ ]:
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

### Appliquer la fonction sur le fichier complet de l'année

In [ ]:
clean_text(year, folder=temp_path)

with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

## Nuage de mots

### Afficher les termes les plus fréquents

In [ ]:
frequencies = Counter(after.split())
print(frequencies.most_common(30))

### Créer, stocker et afficher le nuage de mots

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

# 3. Reconnaissance d'entités nommées avec SpaCy

## Imports

In [ ]:
from collections import defaultdict
import sys
import os
import spacy
from spacy.lang.fr.examples import sentences

In [ ]:
nlp = spacy.load('fr_core_news_lg')

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [ ]:
# Charger le texte
# Rassembler tous les noms des fichiers texte de l'année 1930
txt_path = '../data/txt'
file_names = []
for filename in os.listdir(txt_path):
    if filename.startswith('KB_JB838_1930'):
        file_names.append(os.path.join(txt_path, filename))

# Lire tous les fichiers, sous forme de liste
texts = []
for file_name in file_names:
    with open(file_name, 'r', encoding='utf-8') as file:
        texts.append(file.read())

# Concaténer tous les textes et réduire à 100k caractères pour la calculabilité
all_1930 = '\n'.join(texts)
n = 100000
text = all_1930[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

In [ ]:
# Compter les fréquences des organisations
org = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "ORG" and len(ent.text) > 3:
        org[ent.text] += 1

# Trier et imprimer
sorted_org = sorted(org.items(), key=lambda kv: kv[1], reverse=True)

for org, freq in sorted_org[:50]:
    print(f"{org} apparait {freq} fois dans le corpus")


In [ ]:
# Compter les fréquences des lieux
loc = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "LOC" and len(ent.text) > 3:
        loc[ent.text] += 1

# Trier et imprimer
sorted_loc = sorted(loc.items(), key=lambda kv: kv[1], reverse=True)

for loc, freq in sorted_loc[:50]:
    print(f"{loc} apparait {freq} fois dans le corpus")


# 4. Sentiment analysis

## Textblob

In [ ]:
import sys
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

### Création d'une fonction `get_sentiment`

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    return f"This text is {polarity_str} and {subjectivity_str}."

## Transformers

## Imports

In [ ]:
!pip install tensorflow
!pip install sentencepiece
!pip install transformers

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

## Chargement du modèle

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine", use_pt=True)
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

sentiment_analyser = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

## Analyser le sentiment de 10 phrases du corpus CAMille

In [ ]:
import pandas as pd

# Les phrases
phrases = [
    "C'est une véritable tempête d'acclamations qui salue les souverains à leur retour vers la gare.",
    "Jamais Charleroi n’a vu foule semblable.",
    "Des fenêtres, on jette au passage de la calèche royale une véritable pluie de fleurs et la voiture doit s'arrêter à plusieurs reprises.",
    "A la gare, le Roi et la Reine sont visiblement fatigués.",
    "Ils ont pendant plusieurs heures été exposés à un soleil de plomb, mais leurs figures sont réjouies.",
    "Avant de prendre place dans leur voiture salon, la Reine, puis le Roi remercient vivement le bourgmestre pour la manifestation grandiose de loyalisme qui leur a été donnée et le chargent de transmettre leurs remerciements à la population carolorégienne.",
    "A sept heures moins dix, le train royal quitte la gare, salué par de suprêmes acclamations.",
    "Le service d’ordre a été parfait.",
    "Remercions également M. Lacroix, percepteur principal des télégraphes, qui avait eu l’aimable initiative de mettre un bureau spécial à la disposition des journalistes.",
    "Un homme assassiné dans un sentier, il s’agirait du drame de la jalousie."
]

# Appliquer les fonctions aux phrases et stocker les résultats
results = []
for phrase in phrases:
    sentiment_pos_obj = get_sentiment(phrase)
    sentiment_trans = sentiment_analyser(phrase)
    results.append([phrase, sentiment_pos_obj, sentiment_trans])

# Créer un DataFrame à partir des résultats
df = pd.DataFrame(results, columns=["Phrase", "Sentiment (get_sentiment)", "Sentiment (sentiment_analyser)"])

# Afficher le DataFrame
df
